In [29]:
from glob import glob
import pandas as pd

from sklearn.naive_bayes import MultinomialNB

from nltk import word_tokenize, RegexpTokenizer, TweetTokenizer

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# 데이터 불러오기

In [14]:
train = pd.read_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/train.csv')
train = train[['text','target']]
test = pd.read_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/test.csv')

# preprocessing

In [17]:
#정규표현식을 이용해서 토큰화를 이용해서 .제거
def removing_punctuations(text):
    tokenizer_dots = RegexpTokenizer(r'\w+')
    return " ".join(tokenizer_dots.tokenize(text))

#소문자로 바꾸고 . 제거하는 함수
def cleanner(df):
    tmp = df.copy()
    tmp.text = df.text.apply(lambda x: x.lower())        
    tmp.text = df.text.apply(lambda x: removing_punctuations(x))
    return tmp

train.text = cleanner(train)

# validation data split

In [23]:
x_train, x_text, y_train, y_test = train_test_split(train.text, train.target.values, test_size = 0.3)

# model

In [30]:
def make_model(clf):
    
    pipe_clf = Pipeline(
        [('vect', CountVectorizer(analyzer='word', stop_words='english', tokenizer=word_tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf', clf),
        ]
    )

    pipe_clf = pipe_clf.fit(x_train, y_train)
    y_preds = pipe_clf.predict(x_text)

    print(classification_report(y_test, y_preds))
    
    return pipe_clf

In [31]:
#validation accuracy
pipe_clf = make_model(MultinomialNB())

              precision    recall  f1-score   support

           0       0.80      0.90      0.84      1337
           1       0.82      0.68      0.74       947

    accuracy                           0.81      2284
   macro avg       0.81      0.79      0.79      2284
weighted avg       0.81      0.81      0.80      2284



# submission

In [34]:
clf = pipe_clf.fit(train.text, train.target)
y_preds = clf.predict(test.text)
submission = pd.read_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/sample_submission.csv')
submission.target = y_preds
submission.to_csv('C:/Users/ASUS/Documents/kaggle data/Real or Not NLP with Disaster Tweets/nlp-getting-started/submission.csv', index = False)